In [2]:
"""
Authors: Bibek Aryal, Alex Arnal, Jose Perez
"""
from segmentation.data.data import fetch_loaders
from segmentation.model.frame import Framework
import segmentation.model.functions as fn

import yaml, json, pathlib, warnings, pdb, torch, logging, time
from torch.utils.tensorboard import SummaryWriter
from addict import Dict
import numpy as np

conf = Dict(yaml.safe_load(open('./conf/unet_train.yaml')))
data_dir = pathlib.Path(conf.data_dir)
class_name = conf.class_name
run_name = conf.run_name
processed_dir = data_dir / "processed"
loaders = fetch_loaders(processed_dir, conf.batch_size, conf.use_channels)
loss_fn = fn.get_loss(conf.model_opts.args.outchannels, conf.loss_opts)            
log_dir = pathlib.Path(conf.log_dir) / run_name
frame = Framework(
    loss_fn = loss_fn,
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    device=conf.device
)

Down block with dropout 0.1
Down block with dropout 0.1
Down block with dropout 0.2
Down block with dropout 0.2
Bottleneck block with dropout 0.3
Up block with dropout 0.2
Up block with dropout 0.2
Up block with dropout 0.1
Up block with dropout 0.1


In [3]:
batch = next(iter(loaders['train']))
# Permute for channels first
batch_x = batch[0].permute(0, 3, 1, 2).to(conf.device)
batch_y = batch[1].permute(0, 3, 1, 2).to(conf.device)

print(batch_x.shape, batch_x.dtype)
with torch.no_grad():
    logits = frame.model(batch_x)

torch.Size([24, 3, 512, 512]) torch.float32


In [8]:
print(frame.multi_class)
argmax_logits = torch.argmax(logits, axis=3)

print(torch.sigmoid(logits).shape)
print(torch.nn.Softmax(3)(logits).shape)

True
torch.Size([24, 2, 512, 512])
torch.Size([24, 2, 512, 512])


In [3]:
# with torch.no_grad():
#     tp, fp, fnn = frame.metrics(y_hat, batch_y, conf.loss_masked)
#     metrics = fn.get_metrics(tp, fp, fnn, conf.metrics_opts)
#     fn.print_metrics(conf, metrics, metrics)

# Train | {'Background': {'IoU': 0.49, 'precision': 0.84, 'recall': 0.54}, 'fx': {'IoU': 0.0, 'precision': 0.0, 'recall': 0.0}}


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.